# KMeans Clustering

In [2]:
import pyodbc
import pandas as pd

server = 'dwhunitedoutdoors.database.windows.net'
database = 'DWH_United_Outdoors'
username = 'onderzoeker'
password = 'Wachtwoord1.'
driver = 'ODBC Driver 17 for SQL Server'

connection_string = (
    f'DRIVER={{{driver}}};'
    f'SERVER={server};'
    f'DATABASE={database};'
    f'UID={username};'
    f'PWD={password};'
    'Encrypt=yes;'
    'TrustServerCertificate=no;'
    'Connection Timeout=30;'
)

export_conn = pyodbc.connect(connection_string)
export_cursor = export_conn.cursor()
export_cursor

In [4]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import pandas as pd

query = """
SELECT PURCHASE_ID, PURCHASE_EMPLOYEE_ID 
FROM PURCHASE
"""

df_purchase = pd.read_sql_query(query, export_conn)

encoder = OneHotEncoder()

encoded_data = encoder.fit_transform(df_purchase[['PURCHASE_ID', 'PURCHASE_EMPLOYEE_ID']]).toarray()

df_encoded = pd.DataFrame(encoded_data)

df_encoded.columns = encoder.get_feature_names_out(['PURCHASE_ID', 'PURCHASE_EMPLOYEE_ID'])

k_values = range(2, 28)

silhouette_scores = []

for k in k_values:
   
    kmeans = KMeans(n_clusters=k, random_state=42)
    
    kmeans.fit(df_encoded)
    
    cluster_labels = kmeans.predict(df_encoded)
    
    silhouette_avg = silhouette_score(df_encoded, cluster_labels)
    
    silhouette_scores.append((k, silhouette_avg))

df_silhouette_scores = pd.DataFrame(silhouette_scores, columns=['k', 'silhouette_score'])

df_silhouette_scores = df_silhouette_scores.sort_values('silhouette_score', ascending=False)

df_silhouette_scores


C:\Users\xavie\AppData\Local\Temp\ipykernel_27836\2816147164.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_purchase = pd.read_sql_query(query, export_conn)


,k,silhouette_score
9,11,0.278073
12,14,0.266788
11,13,0.266111
8,10,0.255646
10,12,0.246572
13,15,0.243820
7,9,0.233692
6,8,0.209529
14,16,0.208107
15,17,0.179861


### Beschrijving

De code begint met het importeren van de benodigde libraries voor gegevensvoorverwerking, clustering en evaluatie. Het SQL-query haalt de kolommen `PURCHASE_ID` en `PURCHASE_EMPLOYEE_ID` op uit de `PURCHASE`-tabel en slaat de resultaten op in een DataFrame.

Vervolgens wordt een OneHotEncoder gebruikt om de categorische gegevens (`PURCHASE_ID` en `PURCHASE_EMPLOYEE_ID`) te transformeren naar een binair formaat. De getransformeerde gegevens worden opgeslagen in een nieuwe DataFrame, waarbij de kolommen worden hernoemd om de oorspronkelijke kenmerken te weerspiegelen.

De code definieert een reeks mogelijke waarden voor het aantal clusters (`k`) variërend van 2 tot 27. Voor elk van deze waarden wordt een KMeans-clusteringmodel getraind op de gecodeerde gegevens. Nadat het model is getraind, worden de clusterlabels voorspeld voor de gegevens en wordt de silhouette-score berekend. De silhouette-score meet hoe dicht elk object in een cluster bij de objecten in het naburige cluster staat en helpt bij het evalueren van de kwaliteit van de clustering.

De silhouette-scores voor elk aantal clusters worden opgeslagen in een lijst, die vervolgens wordt omgezet in een DataFrame voor eenvoudige analyse. De DataFrame met silhouette-scores wordt gesorteerd op de silhouette-score in aflopende volgorde om de beste clusteringconfiguratie te identificeren.

Dit proces helpt bij het vinden van het optimale aantal clusters voor de dataset, wat cruciaal is voor segmentatieanalyses en het verkrijgen van inzicht in patronen en groepen binnen de gegevens.

### Belang van dit model

Het belang van dit model bij de opdracht ligt in de clustering- en segmentatieanalyse die het mogelijk maakt. Het gebruik van KMeans clustering in combinatie met de silhouette score helpt om de dataset van aankopen en medewerkers te segmenteren in homogene groepen. Dit heeft verschillende voordelen voor United Outdoors:

    Identificeren van klantsegmenten: Door aankopen te groeperen op basis van verschillende kenmerken, kan United Outdoors verschillende klantsegmenten identificeren. Dit helpt bij het richten van marketingstrategieën en het optimaliseren van productaanbiedingen voor verschillende klantgroepen.

    Optimalisatie van bedrijfsprocessen: Door de clustering van aankoopdata kan United Outdoors patronen ontdekken in het koopgedrag van klanten, wat kan leiden tot verbeterde voorraadbeheer en efficiëntere toeleveringsketens.

    Personeelsanalyse: De clustering van medewerkers op basis van aankoopgegevens kan inzicht bieden in de prestaties van medewerkers, identificeren van training behoeften en optimaliseren van personeelsinzet.

    Strategische besluitvorming: De segmentatie van data ondersteunt de besluitvorming door inzicht te geven in trends en patronen binnen de gefuseerde organisaties. Dit kan helpen bij het bepalen van welke producten moeten worden gepromoot, welke markten moeten worden betreden en hoe middelen het beste kunnen worden toegewezen.

    Data-gedreven inzichten: Het model helpt om data-gedreven inzichten te genereren, wat cruciaal is voor het nemen van strategische beslissingen. Het biedt een kwantitatieve basis voor het begrijpen van klantgedrag en bedrijfsdynamiek.

Door de data effectief te segmenteren en te analyseren, kan United Outdoors zijn concurrentiepositie versterken en beter inspelen op de behoeften van de markt, wat essentieel is in de context van de fusie en uitbreiding.
